In [ ]:
# !pip install langchain-openai
# !pip install pinecone
# !pip install pandas
# !pip install sentence-transformers
# !pip install accelerate
# !pip install hf_xet
# !pip install --upgrade --quiet  transformers --quiet

---
llm으로 json 설명 추가

In [20]:
import os

data_path = './../../datas' 
file_path = os.path.join(data_path, 'flower_data.json')
output_file_path = os.path.join(data_path, 'flower_data_completed.json')

In [21]:
import json

with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

In [5]:
input_datas = [plant['flowNm'] for plant in data]
input_datas

['향나무',
 '나도풍란',
 '군자란',
 '삼나무',
 '수선화',
 '파키라',
 '스토크',
 '아나나스',
 '아마릴리스',
 '우묵사스레피',
 '앵초',
 '브리세아',
 '치자나무',
 '카틀레야',
 '온시디움',
 '크로커스',
 '벤자민고무나무',
 '팔손이',
 '아프리칸 바이올렛',
 '우단일엽',
 '파초',
 '호랑가시나무',
 '사스레피나무',
 '굴거리나무',
 '산호수',
 '꽝꽝나무',
 '구상나무',
 '프리뮬러',
 '괭이눈',
 '칼세올라리아',
 '네펜데스',
 '남천',
 '냉이',
 '동백나무',
 '코르딜리네',
 '디펜바키아',
 '레몬',
 '무스카리',
 '산자고',
 '스노우드롭',
 '스위트피',
 '복수초',
 '산거울',
 '솜나물',
 '아잘레아',
 '안스리움',
 '골담초',
 '에크메아',
 '제라늄',
 '끈끈이주걱',
 '필레아',
 '너도밤나무',
 '탱자나무',
 '프리지아',
 '고란초',
 '석위',
 '렉스베고니아',
 '깽깽이풀',
 '솜다리',
 '우산나물',
 '갯버들',
 '고사리',
 '잉글리쉬데이지',
 '버드나무',
 '벚나무',
 '산수유',
 '서향',
 '시네라리아',
 '히어리',
 '유채',
 '남산제비꽃',
 '제비꽃',
 '조팝나무',
 '진달래',
 '양지꽃',
 '튜립',
 '노루발풀',
 '솜방망이',
 '팬지',
 '황매화',
 '히아신스',
 '피나물',
 '삼지닥나무',
 '노루귀',
 '미선나무',
 '할미꽃',
 '얼레지',
 '각시붓꽃',
 '바람꽃',
 '현호색',
 '앉은부채',
 '개나리',
 '꽃베고니아',
 '라넌큘러스',
 '라일락',
 '애기나리',
 '금낭화',
 '매실나무',
 '천남성',
 '목련',
 '물망초',
 '박태기나무',
 '배나무',
 '명자꽃',
 '꽃댕강나무',
 '살구',
 '수양버들',
 '아디안텀',
 '아이리스',
 '월계수',
 '으름',
 '자작나무',
 '전나무(젓나무)',
 '봄맞이꽃',
 '

In [8]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    template="""
    ### 입력 데이터 ###
    {input_data}

    ### 설명 ###
    당신은 식물 전문가입니다. 입력 데이터의 식물에 대해 다음 4가지 정보를 생성하여 반환하세요.
    - 설명 문장이나 불릿 없이, JSON만 출력하세요.
    - 반드시 항목명(flowNm, desc, difficulty, interior, style)을 그대로 사용하세요.

    0. flowNm: 식물 이름
    1. desc: 꽃의 특징과 키우는 방법을 요약한 소개글 (100자 이내)
    2. difficulty: 키우는 난이도 ('매우 쉬움', '쉬움', '보통', '어려움', '매우 어려움' 중 하나)
    3. interior: 이 식물로 공간을 연출하는 구체적인 방법
    4. style: 이 식물과 가장 잘 어울리는 인테리어 스타일

    ### 출력 형식 ###
    {{
        "flowNm": "",
        "desc": "",
        "difficulty": "",
        "interior": "",
        "style": ""
    }}
    """,
    input_variables=["input_data"],
)

model = ChatOpenAI(
		model='gpt-4o-mini',
		temperature=1
)

chain = prompt | model

In [9]:
input_datas = ['튤립', '사과나무']

In [ ]:
completed_list = []

for idx, item in enumerate(input_datas):
    raw = chain.invoke({'input_data': item})
    
    try:
        obj = json.loads(raw.content)
    except json.JSONDecodeError:
        print(f"{idx}번째 입력에서 JSON 파싱 실패: {item}")
        print("raw:", raw)
        continue

    completed_list.append(obj)

In [15]:
completed_list

[{'flowNm': '튤립',
  'desc': '색상과 모양이 다양한 튤립은 생기가 넘치는 봄의 상징으로, 햇볕을 좋아하고 배수가 잘되는 토양에서 잘 자란다.',
  'difficulty': '쉬움',
  'interior': '화분에 심어 창가나 테이블에 배치하여 밝고 화사한 분위기를 조성할 수 있다.',
  'style': '내추럴'},
 {'flowNm': '사과나무',
  'desc': '사과나무는 아름다운 꽃을 피우며, 햇볕과 잘 배수되는 토양을 선호합니다.',
  'difficulty': '보통',
  'interior': '소형 사과나무를 실내에서 기르면 밝은 공간에 배치하여 자연스러운 분위기를 연출할 수 있습니다.',
  'style': '내추럴'}]

In [23]:
import json

with open(output_file_path, 'w', encoding='utf-8') as f:
    json.dump(completed_list, f, ensure_ascii=False, indent=2)

---
벡터DB

In [ ]:
from pinecone import Pinecone, ServerlessSpec
from dotenv import load_dotenv
import os

load_dotenv()

PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

pc = Pinecone(api_key=PINECONE_API_KEY)

In [ ]:
index_name = 'plants'

pc.create_index(
    name=index_name,
    dimension=384,
    metric='cosine',
    spec=ServerlessSpec(
        cloud='aws',
        region='us-east-1'
    )
)

{
    "name": "plants",
    "metric": "cosine",
    "host": "plants-x7k41h6.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [22]:
import pandas as pd
import json

df = pd.read_json('./../../datas/flower_data.json')

In [23]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

def create_text_for_embedding(row):
    air_cond_text = '공기정화식물이다.' if row['isAirCond'] else ''
    toxic_dog_text = '강아지에게 해롭다.' if row['isToxicToDog'] else ''
    toxic_cat_text = '고양이에게 해롭다.' if row['isToxicToCat'] else ''

    return (
        f"꽃 이름: {row['flowNm']}, "
        f"꽃말: {row['flowLang']}, "
        f"특징: {row['fContent']}, "
        f"용도: {row['fUse']}, "
        f"재배법: {row['fGrow']}, "
        f"종류: {row['fType']}. "
        f"{air_cond_text} "
        f"{toxic_dog_text} "
        f"{toxic_cat_text}"
    ).strip()

df['text_to_embed'] = df.apply(create_text_for_embedding, axis=1)

texts = df['text_to_embed'].tolist()
embeddings = model.encode(texts, show_progress_bar=True)


Batches: 100%|██████████| 12/12 [00:11<00:00,  1.02it/s]


In [ ]:
idx = pc.Index(index_name)

for i, row in df.iterrows():
    metadatas = row.drop(['fSctNm', 'fEngNm', 'fileName1', 'fileName2', 'fileName3', 'publishOrg', 'colors', 'text_to_embed']).to_dict()
    idx.upsert([
        (
            str(row['dataNo']),
            embeddings[i].tolist(),
            metadatas
        )
    ])

In [35]:
query_text = '향기가 좋은 식물 추천해줘'
query_embed = model.encode(query_text).tolist()

results = idx.query(
    vector=query_embed,
    top_k=3,
    include_metadata=True
)
results

{'matches': [{'id': '295',
              'metadata': {'dataNo': 295.0,
                           'fContent': '사랑초(옥살리스, 괭이밥류)라는 이름으로 일반에 잘 알려진 꽃이다. '
                                       '꽃잎이 하트모양을 하고 있으며 해가지면 잎들이 서로 붙어 껴안고 '
                                       '잠자는 모습을 한다. 꽃말보다는 훨씬 더 적극적인 애정을 가진 식물인 '
                                       '것 같다.',
                           'fDay': 21.0,
                           'fGrow': '화단에 두 세포기 심어두면 금방 번진다. 땅속에 있는 뿌리줄기가 옆으로 '
                                    '퍼지면서 마디에서 눈이 나와 포기로 자라기 때문이다. 화분에 심어 '
                                    '포기나누기로 늘려가도 된다. 잡초로도 취급될 만큼 번식이 잘 된다.',
                           'fMonth': 10.0,
                           'fType': '괭이밥과에 속하는 여러해살이풀이다. 세계적으로 약 500종 이상 분포하는 '
                                    '대가족으로 남아프리카와 남미지역에 많이 분포하고 있다. 우리나라에는 '
                                    '애기괭이밥, 큰괭이밥, 괭이밥 세 가지가 있다.',
                           'fUse': '자주색 잎을 띠는 것은 원예용 품종으로 흔히 분화용으로 쓰인다. 정원의 바위 '
                      